### Построение словаря

Напишем словарь, который будет строить эмбеддинги, а также отображение из индексов в слова и обратно

In [1]:
import sys
sys.path.append('../')

In [2]:
from collections import Counter
import fasttext as ft
import json
import os
from thesaurus_parsing.thesaurus_parser import ThesaurusParser
from tqdm import tqdm_notebook as tqdm
from importlib import reload

In [3]:
import vocab as vcb

Этот словарь при построении принимает эмбеддер, который может по токену давать вектор. В изначальном случае это был fasttext от deeppavlov, поэтому завернём его в нужную оболочку для того, чтобы он мог работать, как ожидает код словаря.

In [4]:
class FastTextEmbedder:
    def __init__(self):
        self.model = ft.load_model('../data/models/fasttext_deeppavlov.bin')
        
    def __call__(self, word):
        return self.model.get_word_vector(word)

In [5]:
embedder = FastTextEmbedder()

In [8]:
vcb = reload(vcb)

In [12]:
vocab = vcb.Vocab(embedder=embedder, n_texts=None, cnt_most_common=28)

Widget Javascript not detected.  It may not be installed or enabled properly.


In [15]:
vocab.process_batch(["планета земля", "преобразователь время", "путин"])

[[5226, 6], [2509, 385], [92786, 1]]

Попробуем залить это в файл, потом прочитать заново в новый объект

In [14]:
with open('vocab_lenta_100k.json', 'w') as vocab_file:
    vocab.serialize_to_file(vocab_file)

In [16]:
new_vocab = vcb.Vocab(empty=True)

In [17]:
with open('vocab_lenta_100k.json') as vocab_file:
    new_vocab.construct_from_file(vocab_file)

In [18]:
new_vocab.process_batch(["планета земля", "преобразователь время", "путин"])

[[5226, 6], [2509, 385], [92786, 1]]

In [19]:
matrix, pad_idx = new_vocab.get_embedding_table()

In [20]:
matrix.shape

(121800, 300)

In [21]:
pad_idx

1

Сделаем из этих параметров слой

In [22]:
import torch.nn as nn
import torch

In [23]:
layer = nn.Embedding.from_pretrained(
    torch.FloatTensor(matrix),
    padding_idx=pad_idx,
)

In [24]:
layer

Embedding(121800, 300, padding_idx=1)

In [25]:
for p in layer.parameters():
    print(p)

Parameter containing:
tensor([[-0.0148, -0.0553, -0.1780,  ...,  0.0136, -0.0501,  0.1616],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2767,  0.1283,  0.0490,  ..., -0.4219, -0.0697,  0.3093],
        ...,
        [-0.2515,  0.2553,  0.1998,  ..., -0.0518, -0.3379, -0.5069],
        [-0.2088,  0.2018,  0.1479,  ..., -0.2443, -0.0406, -0.1812],
        [-0.2870,  0.2167, -0.2533,  ..., -0.1063, -0.2958,  0.1576]])


Видим, что паддинг корректно инициализирован нулями